# Image search with autoencoders

Autoencoders are an architecture that have a variety of applications from [denoising data](http://www.jmlr.org/papers/volume11/vincent10a/vincent10a.pdf) to [generative models](https://arxiv.org/1312.6114.pdf).

In this post we will look at how to search for images using autoencoders. The two papers that guide this post are [Semantic Hashing](http://www.cs.utoronto.ca/~rsalakhu/papers/semantic_final.pdf) and [Using Very Deep Autoencoders for Content-Based Image Retrieval](http://www.cs.toronto.edu/~fritz/absps/esann-deep-final.pdf).

# Autoencoders, the gist

The autoencoders considered in this post are simply neural networks that learn the mapping $f:X\mapsto X$. Sounds too easy doesn't it? And why would we want to learn to predict the data we already have?

The key idea behind the architecture of an autoencoder is that a "bottleneck" is placed in the middle of the network. This implicitly forces the network to learn a compressed representation of the input data, which is what we are actually interested in.

More specifically we actually think of the autoencoder as two pieces, the _encoder_ which is the first half of the network mapping $X\in \mathbb{R}^{n}$ to some vector in $\mathbb{R}^{d}$ and the _decoder_ which maps that vector back to $\mathbb{R}^{n}$. The vector output by the encoder is called the _latent representation_ of the data which is what we will use in our image search. Again, at the end of the day all we really care about is the output of the bottleneck.

This picture below should clear up any ambiguity

<img src="./static/autoencoder-architecture.png">

One really nice thing about autoencoders is that the idea described above is very intuitive. Suppose our data $X$ lives in $\mathbb{R}^{n}$ and one of the hidden layers of $f$ outputs a vector of dimmension $d<n$. Then, intuitively it makes sense that if $f$ reconstructs $X$ well after reducing the input in dimmensionality then the output of that hidden layer must contain a lot of information about $X$ in order to reproduce $X$.

# Autoencoders for searching

So how do we apply autoencoders to searching?

Let's consider the use case where we have a set of images $X$, and for a given image $q$ we would like to retrieve all images in $X$ that are "similar" to $q$.

One such way to do this with an autoencoder is to keep a list of key-value pairs of each image $x\in X$ and its encoded representation $\hat{x}$. Recall these codes are simply vectors, so we can compute similarity scores of $\hat{q}$ and each $\hat{x}$, returning the images which scored highest. This is already more efficient than simply computing the similarity scores on the raw values in $X$ in the sense that the encoded representations are much smaller in dimmension. However, both methods are linear searches with time complexity $O(|X|)$, that is the more images we have in our database, the longer the search will take and the less benefit we see in performance from computing the similarity scores on smaller vectors.

It turns out we can still use the codes to perform a faster search in constant time, we just need to add one extra step. We now define the encoder as $\phi:\mathbb{R}^{n}\mapsto\{0,1\}^{d}$ ($d$-bit binary codes). Once again we keep a list of key-value pairs, but this time the key is a $d$-bit integer rather than a real valued vector of dimmension $d$. Then for our query image $q$ we now return all images $x\in X$ such that the hamming distince (number of bits that differ) of $\phi(q)$ and $\phi(x)$ is less than some predetermined threshold $t$.

# A few remarks on building and training the autoencoder

## Learning binary codes

You may be asking how does one constrain the neural network to learn binary codes. It turns out this is actually rather simple. Several techniques, such as adding gaussian noise to the mini-batches, for doing this are discussed in [Semantic Hashing](http://www.cs.utoronto.ca/~rsalakhu/papers/semantic_final.pdf) however the simplest method, described in [here](http://www.cs.toronto.edu/~fritz/absps/esann-deep-final.pdf), is to feed the outputs of the hidden layer which outputs the latent vector into a softmax, round those values up to 1 or down to 0 and then compute the gradient update as if the rounding never occurred.

In the keras we can create a custom layer to this for as follows

In [ ]:
def binarize(x):
    return x + K.stop_gradient(K.round(x) - x)
Binarize = Lambda(binarize, output_shape=lambda x: x, name='encoding')

## Overfitting on the data

Depending on the use case it may be perfectly acceptable to train on the same data we intend to search over.

## Choosing the latent dimmension and threshold

Let's consider the MNIST digits data set as an example of working out a heuristic for choosing the latent dimmension and threshold since that's what we'll be working with below. In the train split provided by keras there are $60,000$ images. If the number of possible binary codes output by the autoencoder is $2^d=N$, then we will have approximately $60,000/N$ images stored per code (assuming the images are distributed uniformly across the codes). In the papers referenced in the beginning of this post the author's found good results when there were about 0.4 items per code so we'll choose $d=17$ which gives us a total of $N=131072$ codes and approximately $0.46$ images per code.

Now, lets choose our threshold as $t=3$, then we have

In [ ]:
import random
%matplotlib inline
from utils import *
X_train, X_test = fetch_data()
show_one(choices(X_train, k=1))

In [ ]:
X_train.shape, X_test.shape

# Training an autoencoder

In [ ]:
from autoencoding import AutoEncoder
INPUT_DIM = X_train.shape[-1]
LATENT_DIM = 17
INTERMEDIATE_DIMS = [1000, 700, 300,  150, 75]
OUTPUT_ACTIVATION = 'sigmoid'
EPOCHS = 125
BATCH_SIZE = 256

In [ ]:
autoencoder = AutoEncoder(
    INPUT_DIM,
    LATENT_DIM,
    INTERMEDIATE_DIMS,
    OUTPUT_ACTIVATION)
autoencoder.summary()

In [ ]:
autoencoder.compile(loss='binary_crossentropy', optimizer='adadelta')

In [ ]:
autoencoder.fit(X_train, X_train,
                epochs=250,
                batch_size=256,
                shuffle=True)

In [ ]:
test_sample = choices(X_test, k=10)
reconstructions = autoencoder.predict(test_sample)
show_side_by_side(test_sample, reconstructions)

In [ ]:
# encoder = autoencoder.encoder
import keras
encoder = keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('encoder').get_output_at(-1))

# Linear search over test images

In [ ]:
def cosine_sim(v1, v2):
    return v1.dot(v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
# define key-value pairs of encoding, image for search
kvps = [(code, image) for code, image in zip(encoder.predict(X_test), X_test)]

In [ ]:
%%time
import random
# pick on image from test set
query_img = random.choice(X_test).reshape(1, 784)

# retrieve similar images
query_code = encoder.predict(query_img)
sims = [(cosine_sim(query_code, stored_code), image) for stored_code, image in kvps]
# get top 10 images
sorted_sims = sorted(sims, reverse=True, key=lambda x: x[0])
top_matches = [img for _, img in sorted_sims[:10]]

show_side_by_side(query_img, top_matches)

In [ ]:
# the codes are binary aren't they?
query_code

# Semantic hashing

In [ ]:
import collections
import numba

# see,
# https://en.wikipedia.org/wiki/Hamming_distance#Algorithm_example
@numba.jit
def hamming_distance(n1, n2):
    # this number is made of each bit in either n1 or n2
    # but not both
    v = n1 ^ n2
    d = 0
    while v != 0:
        # subtracting 1 clears the least bit, a, in v and sets all bits
        # before a which are cleared by the logical &
        # 2^n = sum(2^m for 0 <= m <= n-1)
        d += 1
        v &= v - 1
    return d


class DB:
    def __init__(self, encoder, items=None):
        self.encoder = encoder
        output_dim = self.encoder.output_shape[-1]
        self._z = np.array([2**i for i in range(output_dim)], dtype=np.uint32).transpose()
        self._db = collections.defaultdict(list)
        self.store_items(items)

    def search(self, item, threshold=3, top_n=10):
        key = self._make_keys(item)[0]
        hits = self._find_hits(key, threshold)
        items = self._fetch_items(hits, top_n)
        return items

    @numba.jit
    def _find_hits(self, key, threshold):
        hits = collections.defaultdict(int)
        for other_key in self._db:
            dist = hamming_distance(other_key, key)
            if dist <= threshold:
                hits[other_key] += 2**(threshold-dist)
        return hits

    def _fetch_items(self, hits, top_n):
        out = []
        sorted_hits = sorted(hits.items(), key=lambda x: x[1])
        while len(out) < top_n:
            for key in sorted_hits:
                for item in self._db[key]:
                    out.append(item)
        return out

    def store_items(self, items):
        keys = self._make_keys(items)
        for key, item in zip(keys, items):
            self._db[key].append(item)

    def _make_keys(self, items):
        codes = self.encoder.predict(items)
        return np.dot(codes, self._z).astype(np.uint32)

In [ ]:
%%time
db = DB(encoder, X_train)

In [ ]:
query_img = random.choice(X_train).reshape(1, -1)
res = db.search(query_img, threshold=3)
show_side_by_side(query_img, res)